In [7]:
import os
import sys
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import time
import math
import pandas as pd
import re
import schedule
import folium
import webbrowser
result = pd.DataFrame()

In [8]:
access_key = '39Eri2U1%2BFHElpAAe2Cqjlkbm9Cup%2BTfY%2BQ4I5aAR6Ho5yEyM9Dbng00rv%2B%2FfBVh3m4bsfMnxRYP4yjPBmTy7Q%3D%3D'
access_key2 = 'mRiWAPPWJjiiP+G0At1+VNVWqDae6LZBh0LjLZlwouKuKczV2dGOHYqTeQwfncR+oTBWZp/gIEHXNTzYmjZiOw=='

In [9]:
def remove_tag(content):
   cleanr =re.compile('<.*?>')
   cleantext = re.sub(cleanr, '', content)
   return cleantext

In [10]:
def get_data(roadid,start,end):
    end_point = 'http://data.ulsan.go.kr/rest/roadinformation/LinkTraffic'
    parameters = "?ServiceKey=" + access_key2
    parameters += "&RoadID="+ str(roadid) # 시작페이지
    parameters += "&pageNo=1" # 시작페이지
    parameters += "&numOfRows=10" # 페이지 크기
    url = end_point + parameters
    
    response = urlopen(url).read()
    soup = BeautifulSoup(response, "html.parser")
    
    startnode= soup.find_all('startnodeid')
    for i in range(len(startnode)):
        startnode[i] = remove_tag(str(startnode[i]))
    endnode = soup.find_all("endnodeid")
    for i in range(len(endnode)):
        endnode[i] = remove_tag(str(endnode[i]))
    speed = soup.find_all("speed")
    for i in range(len(speed)):
        speed[i] = remove_tag(str(speed[i]))
    geometry = soup.find_all("geometry")
    for i in range(len(geometry)):
        geometry[i] = remove_tag(str(geometry[i])).replace("\nLineString\n","")[:-1]
    
    result_pd = pd.DataFrame()
    result_pd = pd.DataFrame({"startnode":startnode,"endnode":endnode,"speed":speed,"geometry":geometry})
    result_pd = result_pd.loc[result_pd['startnode'] == str(start), ['startnode','endnode','speed','geometry']].iloc[0]
    result_pd['roadid'] = roadid
    result_pd['time'] = str(datetime.datetime.now())[11:16]
    
    return result_pd

In [11]:
def executing():
    global result
    append_df = pd.DataFrame()
    append_df = append_df.append(get_data(5,1930003300,1930057500))
    append_df = append_df.append(get_data(6,1930003300,1930065500))
    append_df = append_df.append(get_data(8,1930003300,1930003302))
    append_df = append_df.append(get_data(9,1930003100,1930002600))
    append_df = append_df.append(get_data(3338,1930003301,1960085300))
    append_df = append_df.append(get_data(31,1930057500,1930038300))
    append_df = append_df.append(get_data(32,1930065500,1930065100))
    append_df = append_df.append(get_data(34,1930003302,1930003800))
    append_df = append_df.append(get_data(35,1930003300,1930003100))
    append_df = append_df.append(get_data(3339,1960085300,1930003301))
    append_df.reset_index(drop = True,inplace = True)
    result = result.append(append_df[['roadid', 'speed','startnode','endnode','time','geometry']])
    print(datetime.datetime.now())        

    #신복로터리 부근 울산고속도로 node 누락되었음 
    #result.loc[4,'speed'] = '25.0'
    #result.loc[4,'geometry'] = '129.26324587,35.55021166\n129.26016117,35.55222606'
    #result.loc[9,'speed'] = '25.0'
    #result.loc[9,'geometry'] = '129.26043545,35.55251616\n129.26355884,35.55043525'

In [12]:
#한번 수행
executing()
result

2021-12-22 02:40:24.941158


,roadid,speed,startnode,endnode,time,geometry
0,5.0,63.5,1930003300,1930057500,02:40,"129.26367658,35.55007356\n129.2639455,35.54735358"
1,6.0,53.1,1930003300,1930065500,02:40,"129.26371955,35.55002758\n129.2645835,35.54976..."
2,8.0,52.4,1930003300,1930003302,02:40,"129.26378654,35.55003753\n129.26536067,35.5511..."
3,9.0,28.3,1930003100,1930003300,02:40,"129.26201964,35.54708761\n129.26206767,35.5471..."
4,3338.0,null,1930003301,1960085300,02:40,"129.26117452,35.55183553\n129.2605295,35.55226..."
5,31.0,56.7,1930057500,1930003300,02:40,"129.26422367,35.54465467\n129.26425568,35.5448..."
6,32.0,38.5,1930065500,1930003300,02:40,"129.26840332,35.54970867\n129.26724816,35.5497..."
7,34.0,60.2,1930003302,1930003300,02:40,"129.2652726,35.55126758\n129.26369856,35.55011859"
8,35.0,36.2,1930003300,1930003100,02:40,"129.26369555,35.55011553\n129.26362257,35.5500..."
9,3339.0,null,1960085300,1930003301,02:40,"129.25042678,35.55751533\n129.25275366,35.5563..."


In [ ]:
# 5분 마다 값을 받아와서 pd.Dataframe에 저장, interrupt로 stop
schedule.every(5).minutes.do(executing)
while True:
    global result
    schedule.run_pending()
    time.sleep(1)

In [ ]:
# csv 형태로 저장
result.reset_index(drop = True,inplace = True)
result.to_csv('./road_speed_time_geo.csv', sep=',',encoding='utf-8-sig')

In [13]:
# 시각화
roadid = '6'

end_point = 'http://data.ulsan.go.kr/rest/roadinformation/LinkTraffic'
parameters = "?ServiceKey=" + access_key2
parameters += "&RoadID=" + roadid # 시작페이지
parameters += "&pageNo=1" # 시작페이지
parameters += "&numOfRows=100" # 페이지 크기
url = end_point + parameters

response = urlopen(url).read()
soup = BeautifulSoup(response, "html.parser")
soup
def remove_tag(content):
    cleanr =re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', content)
    return cleantext

startnode= soup.find_all('startnodeid')
for i in range(len(startnode)):
    startnode[i] = remove_tag(str(startnode[i]))
endnode = soup.find_all("endnodeid")
for i in range(len(endnode)):
    endnode[i] = remove_tag(str(endnode[i]))
speed = soup.find_all("speed")
for i in range(len(speed)):
    speed[i] = remove_tag(str(speed[i]))
geometry = soup.find_all("geometry")
for i in range(len(geometry)):
    geometry[i] = remove_tag(str(geometry[i])).replace("\nLineString\n","")[:-1]
    

result_pd = pd.DataFrame()
result_pd = pd.DataFrame({"startnode":startnode,"endnode":endnode,"speed":speed,"geometry":geometry})

result_pd


map_line = folium.Map(location=[35.55194166598993, 129.30637237034654], zoom_start=12)

for a in range(len(geometry)):
    r_points = geometry[a].split("\n")
    for i in range(len(r_points)):
            r_points[i] = r_points[i].split(",")
            r_points[i][0] = float(r_points[i][0])
            r_points[i][1] = float(r_points[i][1])

    #위도경도 위치변경
    for i in range(len(r_points)):
        temp_0 = r_points[i][0]
        r_points[i][0] = r_points[i][1]
        r_points[i][1]= temp_0
    
    print(a)
    print(r_points[:2])
    
    #도로 구별을 위해서 부여한항 - 색깔 random을 위한 함수
    import random
    color_ran = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
    
    folium.PolyLine(locations = r_points, tooltip = startnode[a], color=color_ran).add_to(map_line)
    folium.TileLayer('cartodbpositron').add_to(map_line)
map_line

0
[[35.55002758, 129.26371955], [35.54976363, 129.2645835]]
1
[[35.5497335, 129.26545168], [35.54972658, 129.26564662]]
2
[[35.54962955, 129.26839068], [35.54963088, 129.26843927]]
3
[[35.54962037, 129.27144462], [35.54960042, 129.27153562]]
4
[[35.54932141, 129.27293661], [35.54906339, 129.27421567]]
5
[[35.54873407, 129.27552368], [35.54872399, 129.27555567]]
6
[[35.54801716, 129.27743851], [35.54785503, 129.27784842]]
7
[[35.54693824, 129.28006336], [35.54668703, 129.28067082]]
8
[[35.5463226, 129.28231488], [35.5463136, 129.28238689]]
